<a href="https://colab.research.google.com/github/Jaskaran7d5/BCS-secy-recruitment/blob/main/Taskomicron_JaskaranSingh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Logistic regression to predict if you will have heart disease in the next ten years

In [1]:
# Importing required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
df = pd.read_csv("/content/drive/MyDrive/framingham.csv")

In [4]:
print("(No of rows, No of Columns) = ",df.shape)
df.head()

(No of rows, No of Columns) =  (4238, 16)


,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


In [5]:
row,col = df.shape

In [6]:
for column in df:
  df[column].fillna(value=df[column].mean(), inplace=True)

In [7]:
data = df.to_numpy()

In [8]:
data[:,-1]

array([0., 0., 0., ..., 0., 0., 0.])

In [9]:
mean =np.zeros((1,16))
sd=np.zeros((1,16))
for i in range(0,16):
    mean[0,i]= np.mean(data[:,i])
    sd[0,i]=np.std(data[:,i])
data_array_norm = (data - mean)/sd

print(data_array_norm.shape)

(4238, 16)


In [10]:
labels = data_array_norm[:,-1]
x_array = data_array_norm[:,:col-1]
x_train, x_test, y_train, y_test = train_test_split(x_array,labels,test_size=0.15,random_state=42,shuffle=True)    # splitting data into test and train set.

print(x_train.shape,x_test.shape,y_train.shape,y_test.shape)

(3602, 15) (636, 15) (3602,) (636,)


In [11]:
x_train

array([[-0.86715836, -1.23495068,  0.02090466, ..., -0.24862167,
        -0.65527962,  0.48319571],
       [-0.86715836, -0.18491638, -0.97218687, ..., -0.59993225,
         0.84175647, -0.08613301],
       [-0.86715836, -1.23495068, -0.97218687, ..., -1.25341909,
        -0.7384483 ,  0.13283958],
       ...,
       [-0.86715836,  1.21512937,  0.02090466, ..., -0.17492014,
         0.59225045, -0.04233849],
       [ 1.1531919 , -1.23495068,  0.02090466, ..., -0.31986647,
        -0.65527962, -0.87443431],
       [-0.86715836, -1.7016326 ,  0.02090466, ..., -1.26815939,
         1.17443115, -0.08613301]])

In [12]:
x_test

array([[ 1.1531919 ,  1.5651408 , -0.97218687, ...,  0.32625021,
        -0.32260494,  0.17663409],
       [ 1.1531919 , -0.53492781,  1.01399619, ...,  0.53507119,
         0.75858779,  0.04525054],
       [-0.86715836,  0.16509506, -0.97218687, ..., -1.36642809,
        -0.07309892,  0.61457926],
       ...,
       [ 1.1531919 , -0.18491638,  2.00708772, ...,  0.10268892,
        -0.57211095,  0.74596281],
       [ 1.1531919 ,  1.21512937,  0.        , ..., -1.53839831,
        -0.98795431, -0.39269463],
       [ 1.1531919 ,  0.28176554, -0.97218687, ..., -0.2387948 ,
         0.01006975, -0.08613301]])

In [13]:
y_train.any()

True

In [14]:
def sigmoid(x):
        return 1 / (1 + np.exp(-x))

In [15]:
def loss( y_true, y_pred):
        # binary cross entropy
        epsilon = 1e-9
        y1 = y_true * np.log(y_pred + epsilon)
        y2 = (1-y_true) * np.log(1 - y_pred + epsilon)
        return -np.mean(y1 + y2)


In [16]:
def y(x,a,b):

  m,n = x.shape
  y_pred = np.dot(x,a)+b

  assert(y_pred.shape == (m,))
  return y_pred

In [17]:
def gradient(x,a,b,y_true):

  m,n = x.shape
  yp = sigmoid(y(x,a,b))

  da = np.dot(x.T,yp-y_true)/m              # write code to calculate derivative of loss with respect to a
  db = np.sum(yp-y_true)/m              # write code to calculate derivative of loss with respect to b

  assert(da.shape ==(n,))
  return (da,db)

In [18]:
def gradient_descent(x,y_true,learning_rate=0.01,epochs = 10):
  """
  This function perfroms gradient descent and minimizes loss
  input:
  x: [array] the feature vector of shape (m,n)
  y_true: [array] ground truth of shape (m,)

  output:
  loss: [array] of size (epochs,)
  weights: [tuple] (a,b)
  """
  m,n = x.shape
  loss_mse = []                                 # initialize empty list to store loss
  a = np.random.rand(n,)                                      # initialize a- weights and b- bias
  b = 2

  for i in range(epochs):
    # calculate derivative using gradient() function
    # apply gradient descent now to update a and b
    yp = sigmoid(y(x,a,b))
    l_mse = loss(y_true,yp)                               # calculate loss at this point
    loss_mse.append(l_mse)
    da,db = gradient(x,a,b,y_true)
    #print("yp= ",yp)
    #print("y=",y_true)
    print("Epoch ",i+1," Completed!","loss = ",l_mse)
    a =a-da*learning_rate
    b=b-db*learning_rate
  print("Training completed!!")

  assert(a.shape==(n,))

  return (loss_mse,a,b)

In [19]:
aa = np.random.rand(15,)                                      # initialize a- weights and b- bias
bb = 2
yp = sigmoid(y(x_train,aa,bb))
epsilon = 1e-9
y1 = y_train * np.log(yp + epsilon)
y2 = (1-y_train) * np.log(1 - yp + epsilon)
loss(y_train,yp)

1.7990215044489013

In [20]:
epochs = 100              # tweak this!!!
learn_rate = 0.065       # choose learning rate wisely otherwise loss may diverge!!

train_loss,a,b = gradient_descent(x_train,y_train,learn_rate,epochs)

Epoch  1  Completed! loss =  1.8065365126715924
Epoch  2  Completed! loss =  1.7702511658429754
Epoch  3  Completed! loss =  1.7345309618390832
Epoch  4  Completed! loss =  1.6993668461358533
Epoch  5  Completed! loss =  1.6647499133249524
Epoch  6  Completed! loss =  1.6306714114547487
Epoch  7  Completed! loss =  1.5971227455893
Epoch  8  Completed! loss =  1.564095480578964
Epoch  9  Completed! loss =  1.5315813430373715
Epoch  10  Completed! loss =  1.4995722229938173
Epoch  11  Completed! loss =  1.4680601745252988
Epoch  12  Completed! loss =  1.437037416251782
Epoch  13  Completed! loss =  1.406496331044719
Epoch  14  Completed! loss =  1.3764294656937772
Epoch  15  Completed! loss =  1.3468295295621193
Epoch  16  Completed! loss =  1.3176893936556755
Epoch  17  Completed! loss =  1.2890020883395776
Epoch  18  Completed! loss =  1.2607608018239238
Epoch  19  Completed! loss =  1.2329588773864502
Epoch  20  Completed! loss =  1.2055898108821277
Epoch  21  Completed! loss =  1.178

In [21]:
A = np.reshape(a,(15,1))

In [22]:
import math

In [23]:
mean[0:14]

array([[4.29211892e-01, 4.95849457e+01, 1.97894992e+00, 4.94100991e-01,
        9.00308862e+00, 2.96296296e-02, 5.89900897e-03, 3.10523832e-01,
        2.57196791e-02, 2.36721585e+02, 1.32352407e+02, 8.28934639e+01,
        2.58020076e+01, 7.58789238e+01, 8.19667532e+01, 1.51958471e-01]])

In [24]:
def predict(x):
  threshold =0.5
  X = (x-mean[0,0:15])/sd[0,0:15]
  yp = np.dot(X,A)+b
  #print(yp)
  yp = sigmoid(yp)
  res = yp.item()
  return res


In [25]:
xrr = data[:,:col-1]

In [26]:
tesht = np.reshape(xrr[3800,:],(1,15))

In [ ]:
tesht

array([[  0.  ,  48.  ,   1.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,
          0.  , 274.  , 110.  ,  84.  ,  22.51, 110.  ,  78.  ]])

In [ ]:
data.shape

(4238, 16)

In [ ]:
thr=1
ones =[]
zeros=[]
for i in range(0,4238):
  x = predict(np.reshape(xrr[i,:],(1,15)))
  if(data[i,15]==1):
    ones.append(x)
  else:
    zeros.append(x)
ones = np.array(ones)


In [ ]:
np.mean(ones)

0.5011090289009945

In [ ]:
zeros = np.array(zeros)
np.mean(zeros)

0.2692702535503732

In [ ]:
np.std(ones)

0.33005188048299483

In [ ]:
np.std(zeros)

0.2799172563280056

In [ ]:
len(zeros)

3594

In [ ]:
right =0
for i in range(0,4238):
  x = predict(np.reshape(xrr[i,:],(1,15)))
  if x>0.:
    if(data[i,15]==1):
      right = right+1

  else:
    if(data[i,15]==0):
      right =right+1
right

3483